### Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.api import OLS

#SKLearn stuff
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_csv('../../data/kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


### Initial Cleaning

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

The column sqft_basement has 454 '?' values. Used  sqft_living and sqft_above to fill in the values. 

In [4]:
df['sqft_basement'] = df['sqft_living'] - df['sqft_above']

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  int64  
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

#### Drops

Dropping 'id', 'yr_renovated', and 'date'. No use for them and we don't have enough data to use them. 

In [6]:
df = df.drop(['id', 'date', 'yr_renovated'], axis=1) 

#### Outliers?

### Split

Using price as a dependent variable. Assuming our stockholder is a real estate/brokerage company, TBD, who gets a commission on each sale or purchase.

In [7]:
y = df["price"]
X = df.drop("price", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.33)

### Categorical Columns

The column 'waterfront' is a binomial categorical variable. The columns 'bathrooms', 'view', 'zipcode', 'condition', and 'grade' are categorical variables. They will need to be converted into a format that can be modeled. 

Need to drop columns for each. Should make a function to do encoding.

#### Waterfront

The column contains Yes and No values. I'm assuming NAs are Nos so I will be filling them in as such.

In [8]:
X_train['waterfront'].fillna('NO', inplace=True)
X_test['waterfront'].fillna('NO', inplace=True)

In [9]:
waterfront_train = X_train[['waterfront']]
encoder_waterfront = OrdinalEncoder()
encoder_waterfront.fit(waterfront_train)
encoder_waterfront.categories_[0]
waterfront_encoded_train = encoder_waterfront.transform(waterfront_train)
waterfront_encoded_train = waterfront_encoded_train.flatten()
X_train["waterfront"] = waterfront_encoded_train

In [10]:
waterfront_test = X_test[['waterfront']]
encoder_waterfront = OrdinalEncoder()
encoder_waterfront.fit(waterfront_test)
encoder_waterfront.categories_[0]
waterfront_encoded_test = encoder_waterfront.transform(waterfront_test)
waterfront_encoded_test = waterfront_encoded_test.flatten()
X_test["waterfront"] = waterfront_encoded_test

#### View 

The column 'view' has 5 categories; NONE, AVERAGE, GOOD, FAIR, and EXCELLENT. I'm assuming NAs are NONE so I will be filling them in as such.

Created a OneHotEncoder function for the categorical variables. 

In [11]:
#sparse=False

In [12]:
X_train['view'].fillna("NONE", inplace=True)
X_test['view'].fillna("NONE", inplace=True)

In [13]:
X_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15
19709,5,3.75,3330,5042,2.0,0.0,AVERAGE,Average,9 Better,2470,860,2014,98103,47.6497,-122.339,1780,3990
4800,3,2.00,1678,13862,1.0,0.0,NONE,Average,7 Average,1678,0,1994,98030,47.3744,-122.190,1550,11753
13225,4,1.00,1550,15239,1.5,1.0,EXCELLENT,Average,6 Low Average,1370,180,1930,98166,47.4502,-122.378,1790,22047
16111,3,2.50,2760,9471,1.0,0.0,AVERAGE,Average,8 Good,1760,1000,1956,98115,47.6760,-122.272,3040,6765
17711,2,1.50,1400,5810,2.0,0.0,NONE,Average,7 Average,1400,0,1940,98103,47.6843,-122.341,1470,3920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,3,2.50,2230,5800,2.0,0.0,NONE,Average,7 Average,2230,0,2004,98065,47.5308,-121.847,2230,6088
21575,4,2.75,2770,3852,2.0,0.0,NONE,Average,8 Good,2770,0,2014,98178,47.5001,-122.232,1810,5641
5390,4,1.50,1530,9000,1.0,0.0,NONE,Good,6 Low Average,1530,0,1976,98014,47.6492,-121.908,1520,8500
860,1,0.75,380,15000,1.0,0.0,NONE,Average,5 Fair,380,0,1963,98168,47.4810,-122.323,1170,15000


In [14]:
ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
def ohe(df, df2, column):    
    for col in column:
        train = df[[col]]
        ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
        ohe.fit(train)
        encoded_train = ohe.transform(train)
        col_names = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_train = pd.DataFrame(encoded_train,
                                     columns=col_names, index=df.index)
        df = pd.concat([df, encoded_train], axis=1)
        
        test = df2[[col]]
        encoded_test = ohe.transform(test)
        col_names = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_test = pd.DataFrame(encoded_test, 
                                    columns=col_names, index=df2.index)
        df2 = pd.concat([df2, encoded_test], axis=1)
    return df, df2

In [15]:
X_train, X_test = ohe(X_train, X_test, ['view'])

#### Condition 

In [16]:
X_train, X_test = ohe(X_train, X_test, ['condition'])

#### Grade 

In [17]:
X_train, X_test = ohe(X_train, X_test, ['grade'])

#### Zipcode 

In [18]:
X_train, X_test = ohe(X_train, X_test, ['zipcode'])

#### Bathrooms

In [19]:
X_train, X_test = ohe(X_train, X_test, ['bathrooms'])

### Models

In [20]:
df.corr().price.abs().sort_values(ascending=False)

price            1.000000
sqft_living      0.701917
sqft_above       0.605368
sqft_living15    0.585241
bathrooms        0.525906
sqft_basement    0.323799
bedrooms         0.308787
lat              0.306692
floors           0.256804
sqft_lot         0.089876
sqft_lot15       0.082845
yr_built         0.053953
zipcode          0.053402
long             0.022036
Name: price, dtype: float64

#### Model 1

Using our top 4 correlated columns including sqft_living, sqft_above, sqft_living15, and the columns created from encoding 'bathrooms'.

In [21]:
cols = ['sqft_living', 'sqft_above', 'sqft_living15', 
       'bathrooms_x0_0.5', 'bathrooms_x0_0.75', 'bathrooms_x0_1.0',
       'bathrooms_x0_1.25', 'bathrooms_x0_1.5', 'bathrooms_x0_1.75',
       'bathrooms_x0_2.0', 'bathrooms_x0_2.25', 'bathrooms_x0_2.5',
       'bathrooms_x0_2.75', 'bathrooms_x0_3.0', 'bathrooms_x0_3.25',
       'bathrooms_x0_3.5', 'bathrooms_x0_3.75', 'bathrooms_x0_4.0',
       'bathrooms_x0_4.25', 'bathrooms_x0_4.5', 'bathrooms_x0_4.75',
       'bathrooms_x0_5.0', 'bathrooms_x0_5.25', 'bathrooms_x0_5.5',
       'bathrooms_x0_5.75', 'bathrooms_x0_6.0', 'bathrooms_x0_6.25',
       'bathrooms_x0_6.5', 'bathrooms_x0_7.5', 'bathrooms_x0_7.75']

In [22]:
y = y_train
x = X_train.filter(cols, axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     669.9
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:            -2.0017e+05
No. Observations:               14469   AIC:                         4.004e+05
Df Residuals:                   14441   BIC:                         4.006e+05
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              1.367e+06    1.3e+05     10.543      0.000    1.11e+06    1.62e+06
sqft_living         220.1605      5.710     38.560      0.000     208.969     231.352
sqft_above          -39.4669      5.444     -7.249      0.000     -50.138     -28.796
sqft_living15       113.0488      4.819     23.457      0.000     103.602     122.496
bathrooms_x0_0.5   -1.49e+06   1.92e+05     -7.746      0.000   -1.87e+06   -1.11e+06
bathrooms_x0_0.75 -1.406e+06   1.34e+05    -10.499      0.000   -1.67e+06   -1.14e+06
bathrooms_x0_1.0    -1.4e+06   1.29e+05    -10.857      0.000   -1.65e+06   -1.15e+06
bathrooms_x0_1.25 -8.984e+05   1.92e+05     -4.682      0.000   -1.27e+06   -5.22e+05
bathrooms_x0_1.5  -1.435e+06   1.29e+05    -11.146      0.000   -1.69e+06   -1.18e+06
bathrooms_x0_1.75 -1.452e+06   1.28e+05    -11.305      0.000    -1.7e+06    -1.2e+06
bathrooms_x0_2.0  -1.443e+06   1.28e+05    -11.233      0.000   -1.69e+06   -1.19e+06
bathrooms_x0_2.25 -1.454e+06   1.28e+05    -11.335      0.000   -1.71e+06    -1.2e+06
bathrooms_x0_2.5  -1.506e+06   1.28e+05    -11.764      0.000   -1.76e+06   -1.26e+06
bathrooms_x0_2.75 -1.478e+06   1.28e+05    -11.552      0.000   -1.73e+06   -1.23e+06
bathrooms_x0_3.0  -1.438e+06   1.28e+05    -11.240      0.000   -1.69e+06   -1.19e+06
bathrooms_x0_3.25 -1.297e+06   1.28e+05    -10.151      0.000   -1.55e+06   -1.05e+06
bathrooms_x0_3.5  -1.379e+06   1.28e+05    -10.811      0.000   -1.63e+06   -1.13e+06
bathrooms_x0_3.75 -1.195e+06   1.29e+05     -9.251      0.000   -1.45e+06   -9.42e+05
bathrooms_x0_4.0  -1.192e+06   1.29e+05     -9.246      0.000   -1.45e+06    -9.4e+05
bathrooms_x0_4.25  -1.02e+06    1.3e+05     -7.837      0.000   -1.28e+06   -7.65e+05
bathrooms_x0_4.5  -1.182e+06    1.3e+05     -9.129      0.000   -1.44e+06   -9.28e+05
bathrooms_x0_4.75  -6.75e+05   1.38e+05     -4.876      0.000   -9.46e+05   -4.04e+05
bathrooms_x0_5.0  -8.358e+05   1.43e+05     -5.829      0.000   -1.12e+06   -5.55e+05
bathrooms_x0_5.25  -9.45e+05   1.48e+05     -6.374      0.000   -1.24e+06   -6.54e+05
bathrooms_x0_5.5  -4.979e+05   1.67e+05     -2.985      0.003   -8.25e+05   -1.71e+05
bathrooms_x0_5.75 -7.989e+04   1.89e+05     -0.423      0.672    -4.5e+05     2.9e+05
bathrooms_x0_6.0   1.286e+06    1.9e+05      6.783      0.000    9.14e+05    1.66e+06
bathrooms_x0_6.5   -1.68e+06   2.77e+05     -6.074      0.000   -2.22e+06   -1.14e+06
==============================================================================
Omnibus:                     7487.271   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           157554.996
Skew:                           2.019   Prob(JB):                         0.00
Kurtosis:                      18.653   Cond. No.                     1.14e+06
======================

In [23]:
y = y_test
x = X_test.filter(cols, axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     307.4
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:                -98580.
No. Observations:                7128   AIC:                         1.972e+05
Df Residuals:                    7100   BIC:                         1.974e+05
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              1.316e+06   1.29e+05     10.195      0.000    1.06e+06    1.57e+06
sqft_living         242.1008      8.049     30.078      0.000     226.322     257.879
sqft_above          -13.2259      7.866     -1.681      0.093     -28.646       2.194
sqft_living15        84.5144      6.775     12.475      0.000      71.234      97.795
bathrooms_x0_0.5  -1.684e+06   2.76e+05     -6.091      0.000   -2.23e+06   -1.14e+06
bathrooms_x0_0.75 -1.343e+06   1.39e+05     -9.685      0.000   -1.61e+06   -1.07e+06
bathrooms_x0_1.0  -1.364e+06   1.28e+05    -10.658      0.000   -1.61e+06   -1.11e+06
bathrooms_x0_1.25 -1.346e+06   1.62e+05     -8.293      0.000   -1.66e+06   -1.03e+06
bathrooms_x0_1.5  -1.382e+06   1.28e+05    -10.812      0.000   -1.63e+06   -1.13e+06
bathrooms_x0_1.75 -1.429e+06   1.27e+05    -11.218      0.000   -1.68e+06   -1.18e+06
bathrooms_x0_2.0  -1.417e+06   1.27e+05    -11.124      0.000   -1.67e+06   -1.17e+06
bathrooms_x0_2.25 -1.438e+06   1.27e+05    -11.311      0.000   -1.69e+06   -1.19e+06
bathrooms_x0_2.5    -1.5e+06   1.27e+05    -11.851      0.000   -1.75e+06   -1.25e+06
bathrooms_x0_2.75  -1.47e+06   1.27e+05    -11.588      0.000   -1.72e+06   -1.22e+06
bathrooms_x0_3.0  -1.401e+06   1.27e+05    -11.020      0.000   -1.65e+06   -1.15e+06
bathrooms_x0_3.25 -1.318e+06   1.27e+05    -10.385      0.000   -1.57e+06   -1.07e+06
bathrooms_x0_3.5    -1.4e+06   1.26e+05    -11.075      0.000   -1.65e+06   -1.15e+06
bathrooms_x0_3.75 -1.224e+06   1.29e+05     -9.473      0.000   -1.48e+06   -9.71e+05
bathrooms_x0_4.0  -1.303e+06   1.32e+05     -9.907      0.000   -1.56e+06   -1.05e+06
bathrooms_x0_4.25 -1.298e+06   1.37e+05     -9.485      0.000   -1.57e+06   -1.03e+06
bathrooms_x0_4.5  -1.273e+06   1.33e+05     -9.548      0.000   -1.53e+06   -1.01e+06
bathrooms_x0_4.75 -9.717e+05   1.66e+05     -5.865      0.000    -1.3e+06   -6.47e+05
bathrooms_x0_5.0  -1.235e+06   1.52e+05     -8.130      0.000   -1.53e+06   -9.37e+05
bathrooms_x0_5.25 -5.887e+05   1.88e+05     -3.124      0.002   -9.58e+05   -2.19e+05
bathrooms_x0_5.5   -6.04e+05   1.65e+05     -3.656      0.000   -9.28e+05    -2.8e+05
bathrooms_x0_5.75 -1.835e+06   2.76e+05     -6.653      0.000   -2.38e+06   -1.29e+06
bathrooms_x0_6.0  -1.508e+06   1.88e+05     -8.011      0.000   -1.88e+06   -1.14e+06
bathrooms_x0_6.5  -9.035e+05   2.75e+05     -3.283      0.001   -1.44e+06   -3.64e+05
==============================================================================
Omnibus:                     4801.819   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           234310.854
Skew:                           2.628   Prob(JB):                         0.00
Kurtosis:                      30.592   Cond. No.                     7.87e+05
======================

In [24]:
y = y_train
x = X_train.filter(['sqft_living', 'sqft_above', 'sqft_living15', 'bathrooms'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.503
Method:                 Least Squares   F-statistic:                     3656.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:            -2.0099e+05
No. Observations:               14469   AIC:                         4.020e+05
Df Residuals:                   14464   BIC:                         4.020e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1.091e+05   7516.306    -14.517      0.000   -1.24e+05   -9.44e+04
sqft_living     267.3025      5.779     46.258      0.000     255.976     278.629
sqft_above      -47.9250      5.586     -8.580      0.000     -58.874     -36.976
sqft_living15    84.2948      4.981     16.924      0.000      74.532      94.058
bathrooms      5318.7227   4351.562      1.222      0.222   -3210.897    1.38e+04
==============================================================================
Omnibus:                     9859.232   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           331664.501
Skew:                           2.821   Prob(JB):                         0.00
Kurtosis:                      25.766   Cond. No.                     1.32e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
y = y_test
x = X_test.filter(['sqft_living', 'sqft_above', 'sqft_living15', 'bathrooms'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                     1778.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:                -98873.
No. Observations:                7128   AIC:                         1.978e+05
Df Residuals:                    7123   BIC:                         1.978e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -7.54e+04   1.05e+04     -7.186      0.000    -9.6e+04   -5.48e+04
sqft_living     272.0934      8.100     33.591      0.000     256.215     287.972
sqft_above      -14.9743      7.856     -1.906      0.057     -30.373       0.425
sqft_living15    57.7578      6.884      8.390      0.000      44.263      71.253
bathrooms     -1.776e+04   5963.082     -2.979      0.003   -2.95e+04   -6072.399
==============================================================================
Omnibus:                     5637.546   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           382939.147
Skew:                           3.268   Prob(JB):                         0.00
Kurtosis:                      38.308   Cond. No.                     1.30e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.3e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Model 2

Using our top 4 correlated columns including sqft_living, bedrooms, sqft_living15, and the columns created from encoding 'grade' minus '10 Very Good'.

In [26]:
y = y_train
x = X_train.filter(['sqft_living', 'bedrooms', 'sqft_living15', '11 Excellent', '12 Luxury', '13 Mansion', '3 Poor',
       '4 Low', '5 Fair', '6 Low Average', '7 Average', '8 Good', '9 Better'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     5069.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:            -2.0085e+05
No. Observations:               14469   AIC:                         4.017e+05
Df Residuals:                   14465   BIC:                         4.017e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.373e+04   9384.073      2.529      0.011    5340.983    4.21e+04
sqft_living     275.7522      4.076     67.655      0.000     267.763     283.741
bedrooms       -5.63e+04   2944.723    -19.120      0.000   -6.21e+04   -5.05e+04
sqft_living15    66.7019      4.824     13.828      0.000      57.247      76.157
==============================================================================
Omnibus:                     9574.704   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           294439.324
Skew:                           2.730   Prob(JB):                         0.00
Kurtosis:                      24.415   Cond. No.                     1.38e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.38e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [27]:
y = y_test
x = X_test.filter(['sqft_living', 'bedrooms', 'sqft_living15', '11 Excellent', '12 Luxury', '13 Mansion', '3 Poor',
       '4 Low', '5 Fair', '6 Low Average', '7 Average', '8 Good', '9 Better'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     2492.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:                -98783.
No. Observations:                7128   AIC:                         1.976e+05
Df Residuals:                    7124   BIC:                         1.976e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.981e+04   1.26e+04      2.373      0.018    5185.588    5.44e+04
sqft_living     285.0738      5.566     51.218      0.000     274.163     295.985
bedrooms      -5.243e+04   3747.156    -13.993      0.000   -5.98e+04   -4.51e+04
sqft_living15    47.8093      6.664      7.174      0.000      34.746      60.873
==============================================================================
Omnibus:                     5476.763   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           336800.310
Skew:                           3.152   Prob(JB):                         0.00
Kurtosis:                      36.080   Cond. No.                     1.31e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.31e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21597 non-null  float64
 1   bedrooms       21597 non-null  int64  
 2   bathrooms      21597 non-null  float64
 3   sqft_living    21597 non-null  int64  
 4   sqft_lot       21597 non-null  int64  
 5   floors         21597 non-null  float64
 6   waterfront     19221 non-null  object 
 7   view           21534 non-null  object 
 8   condition      21597 non-null  object 
 9   grade          21597 non-null  object 
 10  sqft_above     21597 non-null  int64  
 11  sqft_basement  21597 non-null  int64  
 12  yr_built       21597 non-null  int64  
 13  zipcode        21597 non-null  int64  
 14  lat            21597 non-null  float64
 15  long           21597 non-null  float64
 16  sqft_living15  21597 non-null  int64  
 17  sqft_lot15     21597 non-null  int64  
dtypes: flo

#### Model 3

Using our top 4 correlated columns including sqft_living, bedrooms, 'bathrooms', and the columns created from encoding 'zipcode'.

Note the difference between entering the encoding for 'zipcode' and actually entering 'zipcode'. The encoding gives us a higher R2 value, however, it also gives you a much larger RSME Error.

In [29]:
y = y_train
x = X_train.filter(['sqft_living', 'bathrooms', 'sqft_living15', '98001', '98002',
        '98003', '98004', '98005', '98006', '98007', '98008',
       '98010', '98011', '98014', '98019', '98022', '98023', '98024', '98027',
       '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98038',
       '98039', '98040', '98042', '98045', '98052', '98053', '98055', '98056',
       '98058', '98059', '98065', '98070', '98072', '98074', '98075', '98077',
       '98092', '98102', '98103', '98105', '98106', '98107', '98108', '98109',
       '98112', '98115', '98116', '98117', '98118', '98119', '98122', '98125',
       '98126', '98133', '98136', '98144', '98146', '98148', '98155', '98166',
       '98168', '98177', '98178', '98188', '98198', '98199'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.500
Method:                 Least Squares   F-statistic:                     4826.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:            -2.0103e+05
No. Observations:               14469   AIC:                         4.021e+05
Df Residuals:                   14465   BIC:                         4.021e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1.062e+05   7527.260    -14.104      0.000   -1.21e+05   -9.14e+04
sqft_living     236.8298      4.570     51.827      0.000     227.873     245.787
bathrooms      2372.5129   4348.867      0.546      0.585   -6151.823    1.09e+04
sqft_living15    74.7164      4.866     15.354      0.000      65.178      84.255
==============================================================================
Omnibus:                     9904.852   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           334694.281
Skew:                           2.839   Prob(JB):                         0.00
Kurtosis:                      25.867   Cond. No.                     1.11e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.11e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [30]:
y = y_test
x2 = X_test.filter(['sqft_living', 'bathrooms', 'sqft_living15', '98001', '98002',
        '98003', '98004', '98005', '98006', '98007', '98008',
       '98010', '98011', '98014', '98019', '98022', '98023', '98024', '98027',
       '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98038',
       '98039', '98040', '98042', '98045', '98052', '98053', '98055', '98056',
       '98058', '98059', '98065', '98070', '98072', '98074', '98075', '98077',
       '98092', '98102', '98103', '98105', '98106', '98107', '98108', '98109',
       '98112', '98115', '98116', '98117', '98118', '98119', '98122', '98125',
       '98126', '98133', '98136', '98144', '98146', '98148', '98155', '98166',
       '98168', '98177', '98178', '98188', '98198', '98199'], axis=1)
x2 = sm.add_constant(x2)
model = sm.OLS(y, x2.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                     2368.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:                -98874.
No. Observations:                7128   AIC:                         1.978e+05
Df Residuals:                    7124   BIC:                         1.978e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -7.442e+04   1.05e+04     -7.100      0.000    -9.5e+04   -5.39e+04
sqft_living     262.3246      6.274     41.810      0.000     250.025     274.624
bathrooms     -1.869e+04   5944.056     -3.145      0.002   -3.03e+04   -7042.789
sqft_living15    55.0007      6.732      8.170      0.000      41.804      68.197
==============================================================================
Omnibus:                     5614.850   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           375575.183
Skew:                           3.252   Prob(JB):                         0.00
Kurtosis:                      37.961   Cond. No.                     1.10e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.1e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
scaler = StandardScaler()
scaler.fit(x)
X_train_scaled = scaler.transform(x)
X_test_scaled = scaler.transform(x2)

train_preds = model.predict(X_train_scaled)

test_preds = model.predict(X_test_scaled)

In [32]:
print("Training Metrics:")
# R2
print(f"R2: {r2_score(y_train, train_preds):.3f}")
# MAE
print(f"Mean Absolute Error: {mean_absolute_error(y_train, train_preds):.3f}")
# MSE
print(f"Mean Squared Error: {mean_squared_error(y_train, train_preds):.3f}")
# RMSE - just MSE but set squared=False
print(f"Root Mean Squared Error: {mean_squared_error(y_train, train_preds, squared=False):.3f}")

Training Metrics:
R2: -2.200
Mean Absolute Error: 542085.833
Mean Squared Error: 438463506717.955
Root Mean Squared Error: 662165.770


In [33]:
print("Testing Metrics:")
# R2
print(f"R2: {r2_score(y_test, test_preds):.3f}")
# MAE
print(f"Mean Absolute Error: {mean_absolute_error(y_test, test_preds):.3f}")
# MSE
print(f"Mean Squared Error: {mean_squared_error(y_test, test_preds):.3f}")
# RMSE - just MSE but set squared=False
print(f"Root Mean Squared Error: {mean_squared_error(y_test, test_preds, squared=False):.3f}")

Testing Metrics:
R2: -2.254
Mean Absolute Error: 536365.600
Mean Squared Error: 425404572975.441
Root Mean Squared Error: 652230.460


In [34]:
y = y_train
x = X_train.filter(['sqft_living', 'bathrooms', 'sqft_living15', 'zipcode'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     3803.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:            -2.0085e+05
No. Observations:               14469   AIC:                         4.017e+05
Df Residuals:                   14464   BIC:                         4.017e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -7.894e+07   4.11e+06    -19.198      0.000    -8.7e+07   -7.09e+07
sqft_living     231.3250      4.522     51.157      0.000     222.462     240.189
bathrooms      8779.0750   4307.766      2.038      0.042     335.302    1.72e+04
sqft_living15    93.3866      4.903     19.045      0.000      83.775     102.998
zipcode         803.3739     41.902     19.173      0.000     721.240     885.508
==============================================================================
Omnibus:                    10045.337   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           358630.446
Skew:                           2.881   Prob(JB):                         0.00
Kurtosis:                      26.700   Cond. No.                     1.88e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
y = y_test
x2 = X_test.filter(['sqft_living', 'bathrooms', 'sqft_living15', 'zipcode'], axis=1)
x2 = sm.add_constant(x2)
model = sm.OLS(y, x2.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.509
Method:                 Least Squares   F-statistic:                     1846.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:                -98805.
No. Observations:                7128   AIC:                         1.976e+05
Df Residuals:                    7123   BIC:                         1.977e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -6.85e+07   5.78e+06    -11.850      0.000   -7.98e+07   -5.72e+07
sqft_living     256.9698      6.230     41.245      0.000     244.757     269.183
bathrooms     -1.141e+04   5918.958     -1.927      0.054    -2.3e+04     195.128
sqft_living15    71.5935      6.813     10.509      0.000      58.238      84.949
zipcode         697.3309     58.911     11.837      0.000     581.847     812.815
==============================================================================
Omnibus:                     5738.147   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           413480.481
Skew:                           3.343   Prob(JB):                         0.00
Kurtosis:                      39.708   Cond. No.                     1.89e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.89e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [36]:
scaler = StandardScaler()
scaler.fit(x)
X_train_scaled = scaler.transform(x)
X_test_scaled = scaler.transform(x2)

train_preds = model.predict(X_train_scaled)

test_preds = model.predict(X_test_scaled)

In [37]:
print("Training Metrics:")
# R2
print(f"R2: {r2_score(y_train, train_preds):.3f}")
# MAE
print(f"Mean Absolute Error: {mean_absolute_error(y_train, train_preds):.3f}")
# MSE
print(f"Mean Squared Error: {mean_squared_error(y_train, train_preds):.3f}")
# RMSE - just MSE but set squared=False
print(f"Root Mean Squared Error: {mean_squared_error(y_train, train_preds, squared=False):.3f}")

Training Metrics:
R2: -2.178
Mean Absolute Error: 542085.833
Mean Squared Error: 435384951912.792
Root Mean Squared Error: 659837.065


In [38]:
print("Testing Metrics:")
# R2
print(f"R2: {r2_score(y_test, test_preds):.3f}")
# MAE
print(f"Mean Absolute Error: {mean_absolute_error(y_test, test_preds):.3f}")
# MSE
print(f"Mean Squared Error: {mean_squared_error(y_test, test_preds):.3f}")
# RMSE - just MSE but set squared=False
print(f"Root Mean Squared Error: {mean_squared_error(y_test, test_preds, squared=False):.3f}")

Testing Metrics:
R2: -2.233
Mean Absolute Error: 536491.880
Mean Squared Error: 422709947872.123
Root Mean Squared Error: 650161.478


#### Model 4

Using our top 4 correlated columns including 'sqft_living', 'bedrooms', 'sqft_living15', and the columns created from encoding 'zipcode'.

In [39]:
y = y_train
x = X_train.filter(['sqft_living', 'sqft_living15', 'bedrooms', '98001', '98002',
        '98003', '98004', '98005', '98006', '98007', '98008',
       '98010', '98011', '98014', '98019', '98022', '98023', '98024', '98027',
       '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98038',
       '98039', '98040', '98042', '98045', '98052', '98053', '98055', '98056',
       '98058', '98059', '98065', '98070', '98072', '98074', '98075', '98077',
       '98092', '98102', '98103', '98105', '98106', '98107', '98108', '98109',
       '98112', '98115', '98116', '98117', '98118', '98119', '98122', '98125',
       '98126', '98133', '98136', '98144', '98146', '98148', '98155', '98166',
       '98168', '98177', '98178', '98188', '98198', '98199' ], axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     5069.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:            -2.0085e+05
No. Observations:               14469   AIC:                         4.017e+05
Df Residuals:                   14465   BIC:                         4.017e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.373e+04   9384.073      2.529      0.011    5340.983    4.21e+04
sqft_living     275.7522      4.076     67.655      0.000     267.763     283.741
sqft_living15    66.7019      4.824     13.828      0.000      57.247      76.157
bedrooms       -5.63e+04   2944.723    -19.120      0.000   -6.21e+04   -5.05e+04
==============================================================================
Omnibus:                     9574.704   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           294439.324
Skew:                           2.730   Prob(JB):                         0.00
Kurtosis:                      24.415   Cond. No.                     1.38e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.38e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [40]:
y = y_test
x2 = X_test.filter(['sqft_living', 'sqft_living15', 'bedrooms', '98001', '98002',
        '98003', '98004', '98005', '98006', '98007', '98008',
       '98010', '98011', '98014', '98019', '98022', '98023', '98024', '98027',
       '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98038',
       '98039', '98040', '98042', '98045', '98052', '98053', '98055', '98056',
       '98058', '98059', '98065', '98070', '98072', '98074', '98075', '98077',
       '98092', '98102', '98103', '98105', '98106', '98107', '98108', '98109',
       '98112', '98115', '98116', '98117', '98118', '98119', '98122', '98125',
       '98126', '98133', '98136', '98144', '98146', '98148', '98155', '98166',
       '98168', '98177', '98178', '98188', '98198', '98199' ], axis=1)
x2 = sm.add_constant(x2)
model = sm.OLS(y, x2.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     2492.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:                -98783.
No. Observations:                7128   AIC:                         1.976e+05
Df Residuals:                    7124   BIC:                         1.976e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.981e+04   1.26e+04      2.373      0.018    5185.588    5.44e+04
sqft_living     285.0738      5.566     51.218      0.000     274.163     295.985
sqft_living15    47.8093      6.664      7.174      0.000      34.746      60.873
bedrooms      -5.243e+04   3747.156    -13.993      0.000   -5.98e+04   -4.51e+04
==============================================================================
Omnibus:                     5476.763   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           336800.310
Skew:                           3.152   Prob(JB):                         0.00
Kurtosis:                      36.080   Cond. No.                     1.31e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.31e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [41]:
scaler = StandardScaler()
scaler.fit(x)
X_train_scaled = scaler.transform(x)
X_test_scaled = scaler.transform(x2)

train_preds = model.predict(X_train_scaled)

test_preds = model.predict(X_test_scaled)

In [42]:
print("Training Metrics:")
# R2
print(f"R2: {r2_score(y_train, train_preds):.3f}")
# MAE
print(f"Mean Absolute Error: {mean_absolute_error(y_train, train_preds):.3f}")
# MSE
print(f"Mean Squared Error: {mean_squared_error(y_train, train_preds):.3f}")
# RMSE - just MSE but set squared=False
print(f"Root Mean Squared Error: {mean_squared_error(y_train, train_preds, squared=False):.3f}")

Training Metrics:
R2: -2.254
Mean Absolute Error: 542120.690
Mean Squared Error: 445906896252.050
Root Mean Squared Error: 667762.605


In [43]:
print("Testing Metrics:")
# R2
print(f"R2: {r2_score(y_test, test_preds):.3f}")
# MAE
print(f"Mean Absolute Error: {mean_absolute_error(y_test, test_preds):.3f}")
# MSE
print(f"Mean Squared Error: {mean_squared_error(y_test, test_preds):.3f}")
# RMSE - just MSE but set squared=False
print(f"Root Mean Squared Error: {mean_squared_error(y_test, test_preds, squared=False):.3f}")

Testing Metrics:
R2: -2.305
Mean Absolute Error: 535892.707
Mean Squared Error: 432096556773.439
Root Mean Squared Error: 657340.518


#### Model 4

Using our top 4 correlated columns including 'sqft_living', 'bedrooms', and the columns created from encoding 'zipcode' and 'condition'.

In [44]:
y = y_train
x = X_train.filter(['sqft_living', 'bedrooms', '98001', '98002',
        '98003', '98004', '98005', '98006', '98007', '98008',
       '98010', '98011', '98014', '98019', '98022', '98023', '98024', '98027',
       '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98038',
       '98039', '98040', '98042', '98045', '98052', '98053', '98055', '98056',
       '98058', '98059', '98065', '98070', '98072', '98074', '98075', '98077',
       '98092', '98102', '98103', '98105', '98106', '98107', '98108', '98109',
       '98112', '98115', '98116', '98117', '98118', '98119', '98122', '98125',
       '98126', '98133', '98136', '98144', '98146', '98148', '98155', '98166',
       '98168', '98177', '98178', '98188', '98198', '98199' 
                   'Average', 'Fair', 'Good', 'Poor', 'Very Good'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(y, x.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.506
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     7411.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:            -2.0094e+05
No. Observations:               14469   AIC:                         4.019e+05
Df Residuals:                   14466   BIC:                         4.019e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        8.532e+04   8313.857     10.263      0.000     6.9e+04    1.02e+05
sqft_living   315.5507      2.905    108.628      0.000     309.857     321.245
bedrooms    -5.983e+04   2952.892    -20.260      0.000   -6.56e+04    -5.4e+04
==============================================================================
Omnibus:                     9135.871   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           240618.344
Skew:                           2.599   Prob(JB):                         0.00
Kurtosis:                      22.290   Cond. No.                     9.11e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.11e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [45]:
y = y_test
x2 = X_test.filter(['sqft_living', 'bedrooms', '98001', '98002',
        '98003', '98004', '98005', '98006', '98007', '98008',
       '98010', '98011', '98014', '98019', '98022', '98023', '98024', '98027',
       '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98038',
       '98039', '98040', '98042', '98045', '98052', '98053', '98055', '98056',
       '98058', '98059', '98065', '98070', '98072', '98074', '98075', '98077',
       '98092', '98102', '98103', '98105', '98106', '98107', '98108', '98109',
       '98112', '98115', '98116', '98117', '98118', '98119', '98122', '98125',
       '98126', '98133', '98136', '98144', '98146', '98148', '98155', '98166',
       '98168', '98177', '98178', '98188', '98198', '98199',
                   'Average', 'Fair', 'Good', 'Poor', 'Very Good'], axis=1)
x2 = sm.add_constant(x2)
model = sm.OLS(y, x2.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.508
Method:                 Least Squares   F-statistic:                     3686.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        20:13:15   Log-Likelihood:                -98808.
No. Observations:                7128   AIC:                         1.976e+05
Df Residuals:                    7125   BIC:                         1.976e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        7.328e+04    1.1e+04      6.637      0.000    5.16e+04    9.49e+04
sqft_living   313.1072      3.978     78.717      0.000     305.310     320.904
bedrooms    -5.444e+04   3749.871    -14.519      0.000   -6.18e+04   -4.71e+04
==============================================================================
Omnibus:                     5260.881   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           280653.417
Skew:                           3.003   Prob(JB):                         0.00
Kurtosis:                      33.148   Cond. No.                     8.55e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.55e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [46]:
x.shape

(14469, 3)

In [47]:
x2.shape

(7128, 3)

In [48]:
scaler = StandardScaler()
scaler.fit(x)
X_train_scaled = scaler.transform(x)
X_test_scaled = scaler.transform(x2)

train_preds = model.predict(X_train_scaled)

test_preds = model.predict(X_test_scaled)

In [49]:
print("Training Metrics:")
# R2
print(f"R2: {r2_score(y_train, train_preds):.3f}")
# MAE
print(f"Mean Absolute Error: {mean_absolute_error(y_train, train_preds):.3f}")
# MSE
print(f"Mean Squared Error: {mean_squared_error(y_train, train_preds):.3f}")
# RMSE - just MSE but set squared=False
print(f"Root Mean Squared Error: {mean_squared_error(y_train, train_preds, squared=False):.3f}")

Training Metrics:
R2: -2.259
Mean Absolute Error: 542127.780
Mean Squared Error: 446605872728.683
Root Mean Squared Error: 668285.772


In [50]:
print("Testing Metrics:")
# R2
print(f"R2: {r2_score(y_test, test_preds):.3f}")
# MAE
print(f"Mean Absolute Error: {mean_absolute_error(y_test, test_preds):.3f}")
# MSE
print(f"Mean Squared Error: {mean_squared_error(y_test, test_preds):.3f}")
# RMSE - just MSE but set squared=False
print(f"Root Mean Squared Error: {mean_squared_error(y_test, test_preds, squared=False):.3f}")

Testing Metrics:
R2: -2.310
Mean Absolute Error: 535870.408
Mean Squared Error: 432741228644.193
Root Mean Squared Error: 657830.699
